asd

In [13]:
import pypyodbc as pyodbc
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL


In [14]:
DRIVER_NAME = 'SQL Server'
SERVER_NAME = 'Zotac-B\SQLEXPRESS'
DATABASE_NAME = 'ShopInventoryDB'

connection_string = 'Driver={SQL Server};Server=' + SERVER_NAME + ';Database=' + DATABASE_NAME + ';'


In [17]:
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

In [18]:
query = 'SELECT * FROM Suppliers NATURAL JOIN  '

suppliers = pd.read_sql_query(query, engine)
print(df)


    supplier_id supplier_name           contact_email contact_phone  \
0             1    TechSource  contact@techsource.com   +1-555-1234   
1             2      InnoTech       info@innotech.com   +1-555-5678   
2             3    PixelCraft    sales@pixelcraft.com   +1-555-9876   
3             4      TechWave       info@techwave.com   +1-555-2345   
4             5       EcoTech       sales@ecotech.com   +1-555-6789   
5             6      MegaView    contact@megaview.com   +1-555-3456   
6             7      GigaSync       info@gigasync.com   +1-555-7890   
7             8    TechFusion    sales@techfusion.com   +1-555-4567   
8             9     UltraGrip   contact@ultragrip.com   +1-555-8901   
9            10    CyberHertz     info@cyberhertz.com   +1-555-5678   
10           11     ClearTech     sales@cleartech.com   +1-555-1234   
11           12      ErgoTech    contact@ergotech.com   +1-555-4321   

    delivery_time  
0               5  
1               5  
2               